In [2]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom as pdicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import glob
import cv2
import mahotas as mh

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


In [59]:
img=pdicom.read_file(labels_df['image file path'].iloc[1]) #This snipet checks if the haralick features differ 
                                                        # between left and right sided images
img=((img.pixel_array/65535)*255).astype(np.uint8)
rimg=cv2.flip(img,1)
print(mh.features.haralick(img).mean(0))
print(mh.features.haralick(rimg).mean(0))

[  2.66281334e-01   6.94572947e+00   9.98641531e-01   2.55650261e+03
   6.57182614e-01   8.04108748e+01   1.02190647e+04   4.97135661e+00
   6.35603096e+00   1.38248839e-03   2.19183396e+00  -5.74379135e-01
   9.96995340e-01]
[  2.66281334e-01   6.94572947e+00   9.98641531e-01   2.55650261e+03
   6.57182614e-01   8.04108748e+01   1.02190647e+04   4.97135661e+00
   6.35603096e+00   1.38248839e-03   2.19183396e+00  -5.74379135e-01
   9.96995340e-01]


In [120]:
hr={}

labels_df= pd.read_csv('calc_case_description_test_set.csv',index_col=False)
labels_df.index+=1
labels_df['image file path']='DOI/'+labels_df['image file path']
labels_df['cropped image file path']='DOI/'+labels_df['cropped image file path'].str[:-1]
labels_df['ROI mask file path']='DOI/'+labels_df['ROI mask file path']
redf=pdicom.read_file(labels_df.iloc[i]['image file path'])
print('computing..')    
def ff(x):
    img=pdicom.read_file(x)
    imgpx=((img.pixel_array/65535)*255).astype(np.uint8) #compute haralick features 
    img4=cv2.resize(imgpx,(100,100)).astype(np.uint8)
    return(mh.features.haralick(img4).mean(0))

labels_df['haralick for region of interest '] = labels_df['ROI mask file path'].map(ff)
print('STAGE 1 DONE')
labels_df['haralick for actual breast images '] = labels_df['image file path'].map(ff)
print('STAGE 2 DONE')

computing..
STAGE 1 DONE


In [134]:
sav=labels_df  #make a copy of the "fixed" dataframe 
res=labels_df[['left or right breast','image view','pathology','ROI mask file path','haralick for actual breast images ']][labels_df['pathology']!='BENIGN_WITHOUT_CALLBACK']
res  #res has only features we need 

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path,haralick for region of interest,haralick for actual breast images
1,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,DOI/Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590...,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,"[0.000875522135341, 52.4729019998, 0.860591122...","[0.463845553731, 121.584255943, 0.957431210865..."
2,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,DOI/Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.959...,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,"[0.000271216049763, 71.7687447709, 0.983503879...","[0.254929733668, 140.524037343, 0.972125276246..."
3,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.959...,DOI/Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9...,DOI/Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9...,"[0.000457025996027, 59.3977614529, 0.972418338...","[0.470258025232, 254.551935772, 0.914720798285..."
4,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.959...,DOI/Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9...,DOI/Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9...,"[0.00107313505849, 38.4764302112, 0.9024817213...","[0.470258025232, 254.551935772, 0.914720798285..."
5,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.95...,DOI/Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1....,DOI/Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1....,"[0.000962843412446, 56.28050352, 0.89723084616...","[0.294333383872, 52.7315715233, 0.989539557165..."
6,P_00038,2,RIGHT,MLO,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.95...,DOI/Calc-Test_P_00038_RIGHT_MLO_2/1.3.6.1.4.1....,DOI/Calc-Test_P_00038_RIGHT_MLO_2/1.3.6.1.4.1....,"[0.000519697911951, 71.8304838792, 0.915812269...","[0.294333383872, 52.7315715233, 0.989539557165..."
7,P_00041,1,LEFT,CC,2,calcification,LUCENT_CENTER,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00041_LEFT_CC/1.3.6.1.4.1.9590...,DOI/Calc-Test_P_00041_LEFT_CC_2/1.3.6.1.4.1.95...,DOI/Calc-Test_P_00041_LEFT_CC_2/1.3.6.1.4.1.95...,"[0.00126294934407, 69.8186414652, 0.9868765168...","[0.271023389586, 252.857627538, 0.947422680761..."
8,P_00041,1,LEFT,MLO,2,calcification,LUCENT_CENTER,NaN,2,BENIGN_WITHOUT_CALLBACK,5,DOI/Calc-Test_P_00041_LEFT_MLO/1.3.6.1.4.1.959...,DOI/Calc-Test_P_00041_LEFT_MLO_2/1.3.6.1.4.1.9...,DOI/Calc-Test_P_00041_LEFT_MLO_2/1.3.6.1.4.1.9...,"[0.00348310209109, 53.3639863279, 0.9799068142...","[0.271998330357, 181.34303617, 0.975089786287,..."
9,P_00077,2,LEFT,CC,1,calcification,ROUND_AND_REGULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,3,DOI/Calc-Test_P_00077_LEFT_CC/1.3.6.1.4.1.9590...,DOI/Calc-Test_P_00077_LEFT_CC_1/1.3.6.1.4.1.95...,DOI/Calc-Test_P_00077_LEFT_CC_1/1.3.6.1.4.1.95...,"[0.00181038153997, 11.93315427, 0.969954377085...","[0.293486503078, 64.1319888277, 0.99062767574,..."
10,P_00077,2,LEFT,MLO,1,calcification,ROUND_AND_REGULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,3,DOI/Calc-Test_P_00077_LEFT_MLO/1.3.6.1.4.1.959...,DOI/Calc-Test_P_00077_LEFT_MLO_1/1.3.6.1.4.1.9...,DOI/Calc-Test_P_00077_LEFT_MLO_1/1.3.6.1.4.1.9...,"[0.00306085350086, 10.2679481686, 0.9562315018...","[0.170649086264, 162.603100194, 0.978889326485..."


In [ ]:
#cropped images

import mahotas as mh

x=labels_df.groupby('patient_id')['image file path'].unique()[19]
loc=data_dir+'/'+x[0]
redf=pdicom.read_file(loc[:])
img=((redf.pixel_array/65535)*255).astype(np.uint8)

print(mh.features.haralick(img).mean(0))


In [ ]:
import mahotas as mh                 #NOT USED-----DONT RUN
pat=7
x=labels_df.groupby('patient_id')['image file path'].unique()[pat]
y=labels_df.groupby('patient_id')['cropped image file path'].unique()[pat]
z=labels_df.groupby('patient_id')['ROI mask file path'].unique()[pat]
loc1=data_dir+'/'+x[0]
loc2=data_dir+'/'+y[0]
loc3=data_dir+'/'+z[0]
i1=pdicom.read_file(loc1[:])
i2=pdicom.read_file(loc2[:-1])
i3=pdicom.read_file(loc3[:])
#print(i1,'\n')
#print(i2,'\n')
#print(i3,'\n')
img1=i1.pixel_array
img2=i2.pixel_array
img3=i3.pixel_array

plt.subplot(141).imshow(img1,cmap=plt.cm.bone)
plt.subplot(142).imshow(img2,cmap=plt.cm.bone)
plt.subplot(143).imshow(img3,cmap=plt.cm.bone)
img4=cv2.resize(img1,None,fx=0.25, fy=.25).astype(np.uint8)
plt.subplot(144).imshow(img4,cmap=plt.cm.bone)
plt.show()
print('The cropped image\'s size is',img3.shape,'\n')
print('Size of original to resized image = ',img1.shape,'-->',img4.shape)
img1=img4
res=mh.features.haralick(img1).mean(0)
print(res)

In [ ]:
'''[  2.47427627e-01   9.39696232e+00   9.98748162e-01   3.75460055e+03
   6.97435735e-01   9.62242458e+01   1.50090052e+04   5.05213933e+00
   6.23526174e+00   1.52376333e-03   1.96511715e+00  -6.29075092e-01
   9.98328112e-01] =>1/2

[  2.46762020e-01   2.53850976e+01   9.96603323e-01   3.73942522e+03
   6.59817510e-01   9.60958337e+01   1.49323158e+04   5.06199936e+00
   6.46313755e+00   1.39633891e-03   2.23994539e+00  -5.79466160e-01
   9.97410027e-01] =>1/4 
'''


In [195]:
# maps malignant and benign to variables 
d = {'BENIGN': [0,1], 'MALIGNANT': [1,0]}
res['pathology']=res['pathology'].map(d)

,left or right breast,image view,pathology,ROI mask file path,haralick for actual breast images
1,LEFT,CC,"[0, 1]",DOI/Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.95...,"[0.463845553731, 121.584255943, 0.957431210865..."
2,LEFT,MLO,"[0, 1]",DOI/Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9...,"[0.254929733668, 140.524037343, 0.972125276246..."
15,RIGHT,CC,"[0, 1]",DOI/Calc-Test_P_00100_RIGHT_CC_1/1.3.6.1.4.1.9...,"[0.299322892807, 259.873120345, 0.977703523677..."
16,RIGHT,MLO,"[0, 1]",DOI/Calc-Test_P_00100_RIGHT_MLO_1/1.3.6.1.4.1....,"[0.22729081202, 647.521420008, 0.947073748152,..."
17,RIGHT,CC,"[1, 0]",DOI/Calc-Test_P_00127_RIGHT_CC_1/1.3.6.1.4.1.9...,"[0.277620666313, 551.124392664, 0.915993669862..."
18,RIGHT,MLO,"[1, 0]",DOI/Calc-Test_P_00127_RIGHT_MLO_1/1.3.6.1.4.1....,"[0.177757694618, 613.230703245, 0.947678030444..."
19,LEFT,MLO,"[0, 1]",DOI/Calc-Test_P_00132_LEFT_MLO_1/1.3.6.1.4.1.9...,"[0.192734090894, 644.42290302, 0.961838424675,..."
28,LEFT,CC,"[0, 1]",DOI/Calc-Test_P_00141_LEFT_CC_1/1.3.6.1.4.1.95...,"[0.239711942993, 632.694930364, 0.909278925023..."
29,LEFT,MLO,"[0, 1]",DOI/Calc-Test_P_00141_LEFT_MLO_1/1.3.6.1.4.1.9...,"[0.141582010281, 572.122584175, 0.956593285671..."
30,RIGHT,MLO,"[1, 0]",DOI/Calc-Test_P_00150_RIGHT_MLO_1/1.3.6.1.4.1....,"[0.0619914846466, 139.020458117, 0.98610421878..."


In [129]:
##Adding the view as an input to the NN
for i in range(1,len(labels_df)+1):
    if (labels_df['image view'][i]=='CC'):
        labels_df['haralick for actual breast images '][i]=np.append(labels_df['haralick for actual breast images '][i],1)
    elif (labels_df['image view'][i]=='MLO'):
        labels_df['haralick for actual breast images '][i]=np.append(labels_df['haralick for actual breast images '][i],0)   

C:\Users\Sumanth\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Sumanth\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [198]:
#training and testing split 
train=res.sample(frac=0.9,random_state=200)
test=res.drop(train.index)
test_labels=test['pathology'].tolist()
train_labels=train['pathology'].tolist()
test_features=test['haralick for actual breast images '].tolist()
train_features=train['haralick for actual breast images '].tolist()
print(len(test))
print(len(train))

26
233


In [201]:
#Neural Network model using Haralick features
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data


n_nodes_hl1 = 64
n_nodes_hl2 = 32
n_nodes_hl3 = 16
n_nodes_hl4 = 8

n_classes = 2
batch_size = 10
hm_epochs = 100

x = tf.placeholder('float')
y = tf.placeholder('float')

hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(train_features[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}
hidden_4_layer = {'f_fum':n_nodes_hl4,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_nodes_hl4])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl4]))}
output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


# Nothing changes
def neural_network_model(data):

    l1 = tf.add(tf.matmul(data,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

    return output

def train_neural_network(x):
	prediction = neural_network_model(x)  
	cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
	optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

	with tf.Session() as sess:
		sess.run(tf.initialize_all_variables())
	    
		for epoch in range(hm_epochs):
			epoch_loss = 0
			i=0
			while i < len(train_features):
				start = i
				end = i+batch_size
				batch_x = np.array(train_features[start:end])
				batch_y = np.array(train_labels[start:end])

				_, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
				                                              y: batch_y})
				epoch_loss += c
				i+=batch_size
				
			print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
		correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
		accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

		print('Accuracy:',accuracy.eval({x:test_features, y:test_labels}))
	    
train_neural_network(x)

Epoch 1 completed out of 100 loss: 20628529.2344
Epoch 2 completed out of 100 loss: 6028844.34766
Epoch 3 completed out of 100 loss: 718573.674805
Epoch 4 completed out of 100 loss: 389101.518799
Epoch 5 completed out of 100 loss: 266719.768066
Epoch 6 completed out of 100 loss: 221005.0979
Epoch 7 completed out of 100 loss: 256427.782471
Epoch 8 completed out of 100 loss: 214499.399414
Epoch 9 completed out of 100 loss: 220390.409668
Epoch 10 completed out of 100 loss: 224297.774902
Epoch 11 completed out of 100 loss: 228438.483154
Epoch 12 completed out of 100 loss: 218169.950684
Epoch 13 completed out of 100 loss: 229348.062744
Epoch 14 completed out of 100 loss: 207557.374023
Epoch 15 completed out of 100 loss: 206772.229858
Epoch 16 completed out of 100 loss: 226234.661095
Epoch 17 completed out of 100 loss: 208986.526733
Epoch 18 completed out of 100 loss: 215123.038696
Epoch 19 completed out of 100 loss: 168446.597626
Epoch 20 completed out of 100 loss: 163151.938232
Epoch 21 co